In [68]:
import nltk

# Descargar los recursos necesarios
nltk.download('stopwords')

# Descargar el tokenizer 'punkt'
nltk.download('punkt')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cjsal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cjsal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [71]:
import nltk
nltk.download('punkt')

# Verificar que la tokenización funcione
text_sample = "This is a sample text for testing."
tokens = nltk.word_tokenize(text_sample)
print(tokens)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cjsal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\cjsal/nltk_data'
    - 'c:\\Users\\cjsal\\AppData\\Local\\Programs\\Python\\Python310\\nltk_data'
    - 'c:\\Users\\cjsal\\AppData\\Local\\Programs\\Python\\Python310\\share\\nltk_data'
    - 'c:\\Users\\cjsal\\AppData\\Local\\Programs\\Python\\Python310\\lib\\nltk_data'
    - 'C:\\Users\\cjsal\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:/nltk_data'
    - 'C:/nltk_data'
    - 'C:/nltk_data'
    - 'C:/nltk_data'
**********************************************************************


In [63]:
import pandas as pd

# Load the dataset
df = pd.read_excel('Corpus-Agro.xlsx')

# Show the first few rows of the dataset
df.head()


,URL de Documento,Resumen
0,https://faolex.fao.org/docs/pdf/vie78248.pdf,"This Decree provides for the functions, tasks,..."
1,https://faolex.fao.org/docs/pdf/vie95279.pdf,This Decision approves the Scheme on the devel...
2,https://faolex.fao.org/docs/pdf/ita121539.pdf,This Regional Act sets out the legislative fra...
3,https://faolex.fao.org/docs/pdf/cro126979.pdf,This Regulation amends various provisions of t...
4,https://faolex.fao.org/docs/pdf/chn137120C.pdf...,This Law is enacted for the purposes of guaran...


In [64]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text.lower())
    # Remove stopwords and punctuation
    tokens = [word for word in tokens if word.isalnum() and word not in stopwords.words('english')]
    return tokens


In [65]:
# Preprocess each summary and store results in a new list
processed_resumen = [preprocess_text(resumen) for resumen in df['Resumen']]

# Add the processed summaries as a new column to the DataFrame
df['Processed_Resumen'] = processed_resumen

# Show the processed corpus
df[['URL del Documento', 'Processed_Resumen']].head()


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\cjsal/nltk_data'
    - 'c:\\Users\\cjsal\\AppData\\Local\\Programs\\Python\\Python310\\nltk_data'
    - 'c:\\Users\\cjsal\\AppData\\Local\\Programs\\Python\\Python310\\share\\nltk_data'
    - 'c:\\Users\\cjsal\\AppData\\Local\\Programs\\Python\\Python310\\lib\\nltk_data'
    - 'C:\\Users\\cjsal\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:/nltk_data'
    - 'C:/nltk_data'
    - 'C:/nltk_data'
    - 'C:/nltk_data'
**********************************************************************


In [ ]:
from gensim.models import Word2Vec

# Train Word2Vec model on the processed summaries
model = Word2Vec(sentences=df['Processed_Resumen'], vector_size=100, window=5, min_count=1, workers=4)

# Show the vocabulary
model.wv.index_to_key


In [ ]:
# Request a query from the user
query = input("Enter your search query: ")

# Preprocess the query
processed_query = preprocess_text(query)

# Create a vector for the query
query_vector = sum(model.wv[word] for word in processed_query if word in model.wv)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Initialize a list to hold similarities
similarities = []

# Calculate cosine similarity between the query vector and each summary vector
for resumen in df['Processed_Resumen']:
    if resumen:  # Ensure resumen is not empty
        resumen_vector = sum(model.wv[word] for word in resumen if word in model.wv)
        similarity = cosine_similarity([query_vector], [resumen_vector])
        similarities.append(similarity[0][0])
    else:
        similarities.append(0)  # Handle empty summaries

# Add similarities to the DataFrame
df['Similarity'] = similarities


In [ ]:
# Show the top N results
N = 5  # Number of top results to show
top_results = df.nlargest(N, 'Similarity')

# Display the results
print(top_results[['URL del Documento', 'Similarity']])
